##  Sentinel-3 SLSTR composites

In [1]:
false_color_infrared = dict([('id', 'false_color_infrared'),
                       ('title', 'False Color Infrared (S3, S2, S1)'),
                       ('abstract', 'False Color Infrared (S3, S2, S1)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

In [2]:
false_color_1 = dict([('id', 'false_color_1'),
                       ('title', 'False Color 1 (S5, S3, S2)'),
                       ('abstract', 'False Color 1 (S5, S3, S2)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

In [3]:
aoi_wkt = 'POLYGON((123.311 68.552,123.311 69.975,127.683 69.975,127.683 68.552,123.311 68.552))'
aoi_wkt = 'POLYGON((123.618 67.136,123.618 69.581,128.716 69.581,128.716 67.136,123.618 67.136))' # ok
aoi_wkt = 'POLYGON((121.948 66.566,121.948 70.14,130.518 70.14,130.518 66.566,121.948 66.566))' # ok
aoi_wkt = 'POLYGON((120.059 65.586,120.059 70.583,132.583 70.583,132.583 65.586,120.059 65.586))'

### STAC catalog with the SLSTR acquisition staged-in

In [5]:
input_catalog = '/workspace/data/catalog.json'

### Workflow

#### Import the packages

In [6]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
import sys
sys.path.append(os.path.join(os.environ['PREFIX'], 'conda-otb/lib/python'))
os.environ['OTB_APPLICATION_PATH'] = os.path.join(os.environ['PREFIX'], 'conda-otb/lib/otb/applications')
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
import otbApplication
import gdal
from helpers import *

from shapely.geometry import box, shape, mapping
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType
from IPython.display import GeoJSON
import imageio
from io import BytesIO
from PIL import Image
from base64 import b64encode
from ipyleaflet import Map, ImageOverlay

os.environ['_JAVA_OPTIONS'] = '-Xms1g -Xmx1g'

gdal.UseExceptions()

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [9]:
collection = next(cat.get_children())

In [10]:
item = next(collection.get_items())

In [11]:
item.properties['eop:orbitDirection']

'DESCENDING'

In [12]:
if item.properties['eop:orbitDirection'] != 'DESCENDING':
    ciop.log('ERROR','Product cannot be used as input')
    raise Exception('Use products with descending orbit direction')

### Map

In [13]:
GeoJSON([{'type': 'Feature', 
          'properties': {},
          'geometry': mapping(loads(aoi_wkt))}])

<IPython.display.GeoJSON object>

### Import Sentinel-3 SLSTR product

In [14]:
operators = ['Read',
             'Subset',
             'Rad2Refl',
             'Resample',
             'Reproject',
             'Write']

In [15]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3_SLSTRL1B_500m'

subset = dict()

subset['geoRegion'] = aoi_wkt 

rad2refl = dict()

rad2refl['sensor'] = 'SLSTR_500m'
rad2refl['copyTiePointGrids'] = 'true'
rad2refl['copyFlagBandsAndMasks'] = 'true'
rad2refl['copyNonSpectralBands'] = 'true'

resample = dict()
resample['referenceBandName'] = 'S1_reflectance_an'

reproject = dict()
reproject['crs'] = 'EPSG:32653'

write = dict()
write['file'] = 's3_slstr'
write['clearCacheAfterRowWrite'] = 'true'

In [ ]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Subset=subset,
           Rad2Refl=rad2refl,
           Resample=resample,
           Reproject=reproject,
           Write=write)

### RGB Composites

In [ ]:
date_format = '%Y%m%dT%H%m%S'

output_startdate = item.datetime
output_stopdate = item.datetime

In [ ]:
composites = dict()

composites['S3 SLSTR False color Infrared'] = {'bands': 'S3_reflectance_an,S2_reflectance_an,S1_reflectance_an',
                                               'create': True if (false_color_infrared['value'] == 'Yes') else False,
                                               'hfact': 3.0}

composites['S3 SLSTR False color 1'] = {'bands': 'S5_reflectance_an,S3_reflectance_an,S2_reflectance_an',
                                        'create': True if (false_color_1['value'] == 'Yes') else False,
                                        'hfact': 3.0}


In [ ]:
for k, v in composites.items():
    
    if not v['create']:
        
        continue
    
    bands = [os.path.join(write['file'] + '.data',  '{}.img'.format(band)) for band in (composites[k]['bands'].split(',') +  
                                                                         ['cloud_an',
                                                                          'confidence_an',
                                                                          'S2_exception_an'])]
    
    print(bands)
    
    ds = gdal.Open(bands[0])

    geo_transform = ds.GetGeoTransform()
    projection_ref = ds.GetProjectionRef()
    
    ds = None
    
    s3_rgb_data = read_s3(bands)
    
    red = s3_rgb_data[:,:,0]
    green = s3_rgb_data[:,:,1]
    blue = s3_rgb_data[:,:,2]
    cloud = s3_rgb_data[:,:,3]
    confidence = s3_rgb_data[:,:,4]
    exception = s3_rgb_data[:,:,5]

    date_format = '%Y%m%dT%H%m%S'
    
    output_name = '-'.join([k.replace(' ', '-').upper(), 
                            output_startdate.strftime(date_format), 
                            output_startdate.strftime(date_format)])
    
    s3_rgb_composite(red, 
                     green,
                     blue, 
                     cloud, 
                     confidence,
                     exception,
                     geo_transform,
                     projection_ref, 
                     output_name + '.tif',
                     v['hfact'])
    
    # PNG
    gdal.Translate('{}.png'.format(output_name), 
                   '{}.tif'.format(output_name), 
                   format='PNG')

    os.remove('{}.png.aux.xml'.format(output_name))
    

### Clean-up

In [ ]:
shutil.rmtree('{}.data'.format(write['file']))

In [ ]:
os.remove('{}.dim'.format(write['file']))

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.